In [1]:
!pip install efficientnet-pytorch groq python-dotenv torch torchvision pillow



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import torch
import torch.nn.functional as F
from PIL import Image
from efficientnet_pytorch import EfficientNet
from typing import Dict, Optional
import numpy as np
from dotenv import load_dotenv


In [3]:
from groq import Groq

load_dotenv()
client = Groq()

print("✅ Groq Client Initialized Securely")


✅ Groq Client Initialized Securely


In [4]:
DISEASE_KNOWLEDGE = """

Pepper Bell Bacterial Spot:
- Symptoms: Small dark water-soaked spots on leaves and fruits, yellow halos, leaf drop
- Cause: Xanthomonas bacteria
- Organic Control: Neem oil, copper spray
- Chemical Control: Copper fungicide, Streptomycin
- Prevention: Disease-free seeds, crop rotation, avoid overhead irrigation

Pepper Bell Healthy:
- Condition: Plant shows normal green leaves, good growth, no infections
- Prevention: Proper irrigation, balanced fertilizer, pest monitoring

Potato Early Blight:
- Symptoms: Brown concentric rings on lower leaves, yellowing, leaf drop
- Cause: Fungus (Alternaria solani)
- Organic Control: Neem oil, compost tea
- Chemical Control: Mancozeb, Chlorothalonil
- Prevention: Crop rotation, remove infected debris

Potato Healthy:
- Condition: Normal leaf color, strong stems, healthy tuber development
- Prevention: Proper drainage, disease-free seed tubers, balanced nutrition

✅ Potato Late Blight:
- Symptoms: Dark water-soaked lesions on leaves, white mold under leaves, tuber rot
- Cause: Phytophthora infestans
- Organic Control: Copper oxychloride, neem oil, baking soda spray
- Chemical Control: Metalaxyl, Mancozeb, Chlorothalonil
- Prevention: Crop rotation, remove infected plants, resistant varieties

Tomato Bacterial Spot:
- Symptoms: Small dark lesions on leaves and fruit, yellow margins
- Cause: Xanthomonas bacteria
- Organic Control: Neem oil, copper soap
- Chemical Control: Copper fungicides
- Prevention: Clean seeds, avoid wet foliage

Tomato Early Blight:
- Symptoms: Brown rings on leaves, leaf yellowing
- Cause: Alternaria solani
- Organic Control: Neem oil, compost spray
- Chemical Control: Mancozeb, Chlorothalonil
- Prevention: Crop rotation, mulch soil

Tomato Healthy:
- Condition: Bright green leaves, healthy fruit, no lesions
- Prevention: Balanced nutrients, regular pruning, pest control

Tomato Late Blight:
- Symptoms: Black greasy lesions, rapid plant collapse
- Cause: Phytophthora infestans
- Organic Control: Copper fungicide
- Chemical Control: Metalaxyl, Mancozeb
- Prevention: Avoid excess moisture, resistant hybrids

Tomato Leaf Mold:
- Symptoms: Yellow patches on upper leaf, olive-green mold under leaf
- Cause: Passalora fulva fungus
- Organic Control: Baking soda spray, neem oil
- Chemical Control: Chlorothalonil
- Prevention: Good air circulation, reduce humidity

Tomato Septoria Leaf Spot:
- Symptoms: Small brown spots with gray centers, leaf drop
- Cause: Septoria lycopersici fungus
- Organic Control: Neem oil, compost tea
- Chemical Control: Fungicides (Captan, Mancozeb)
- Prevention: Remove infected leaves, rotate crops

Tomato Spider Mites:
- Symptoms: Yellow stippling on leaves, webbing, weak plants
- Cause: Two-spotted spider mite pest
- Organic Control: Neem oil, water spray, insecticidal soap
- Chemical Control: Abamectin
- Prevention: Maintain humidity, remove weeds

Tomato Target Spot:
- Symptoms: Dark circular lesions with yellow borders
- Cause: Corynespora cassiicola fungus
- Organic Control: Neem oil, bio-fungicide
- Chemical Control: Chlorothalonil
- Prevention: Spacing plants, avoid leaf wetness

Tomato Mosaic Virus:
- Symptoms: Mottled leaves, distorted growth
- Cause: Viral infection
- Control: No cure — remove infected plants
- Prevention: Disease-free seeds, disinfect tools

Tomato Yellow Leaf Curl Virus:
- Symptoms: Yellow curled leaves, stunted growth
- Cause: Virus transmitted by whiteflies
- Organic Control: Neem oil for whitefly control
- Chemical Control: Imidacloprid
- Prevention: Insect netting, remove infected plants

"""


In [5]:
INSECT_KNOWLEDGE = """
Aphids:
- Damage: Suck sap from leaves
- Organic Control: Neem oil, soap spray

Whiteflies:
- Damage: Transmit plant viruses
- Organic Control: Neem oil
- Control: Yellow sticky traps, Imidacloprid

Spider Mites:
- Damage: Yellow speckles, webbing on leaves
- Organic Control: Neem oil, water spray
- Chemical Control: Abamectin
"""


In [6]:
class AgriculturalRAG:
    def __init__(self, client):
        self.client = client
        self.knowledge_base = ""

        # ✅ Safe fallback model list (auto-switch if one fails)
        self.models = [
            "llama-3.1-8b-instant",   # ✅ Fast & currently stable
            "gemma2-9b-it",           # ✅ Stable instruction model
        ]

    def load_knowledge_base(self, disease_knowledge, insect_knowledge):
        self.knowledge_base = f"""
        DISEASE KNOWLEDGE:
        {disease_knowledge}

        INSECT KNOWLEDGE:
        {insect_knowledge}
        """
        print("✅ Knowledge Base Loaded into RAG")

    def query(self, question, verbose=False):
        prompt = f"""
You are an expert agricultural plant pathologist.

Use the following knowledge to answer carefully and accurately.

KNOWLEDGE BASE:
{self.knowledge_base}

QUESTION:
{question}

Give:
- Symptoms
- Cause
- Organic Treatment
- Chemical Treatment
- Prevention
"""

        last_error = None

        # ✅ Try models one by one until one works
        for model_name in self.models:
            try:
                if verbose:
                    print(f"🔄 Trying Groq model: {model_name}")

                response = self.client.chat.completions.create(
                    model=model_name,
                    messages=[
                        {"role": "system", "content": "You are a professional agricultural disease expert."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.2
                )

                answer = response.choices[0].message.content

                return {
                    "answer": answer,
                    "debug": {
                        "model_used": model_name,
                        "sources": ["Groq LLM + Custom Knowledge Base"]
                    }
                }

            except Exception as e:
                last_error = str(e)
                if verbose:
                    print(f"❌ Model failed: {model_name}")

        # ✅ If all models fail
        return {
            "answer": f"⚠️ All Groq models failed. Last error:\n{last_error}",
            "debug": {"model_used": None, "sources": []}
        }


In [7]:
rag = AgriculturalRAG(client=client)
rag.load_knowledge_base(DISEASE_KNOWLEDGE, INSECT_KNOWLEDGE)

print("✅ RAG System Ready")


✅ Knowledge Base Loaded into RAG
✅ RAG System Ready


In [8]:
class IntegratedPlantDiseaseSystem:
    
    def __init__(self, cnn_model_path: str, rag_system, confidence_threshold: float = 0.75):
        self.rag = rag_system
        self.confidence_threshold = confidence_threshold
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        checkpoint = torch.load(cnn_model_path, map_location=self.device)
        
        self.class_names = checkpoint["classes"]
        self.cnn_model = EfficientNet.from_pretrained(
            'efficientnet-b0', 
            num_classes=len(self.class_names)
        )
        self.cnn_model.load_state_dict(checkpoint["model"])
        self.cnn_model.to(self.device)
        self.cnn_model.eval()
        
        from torchvision import transforms
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                [0.485, 0.456, 0.406], 
                [0.229, 0.224, 0.225]
            )
        ])
        
        print("✅ Integrated system initialized")
        print("CNN Classes:", len(self.class_names))
        print("Confidence Threshold:", confidence_threshold)
        print("Device:", self.device)
    

    def predict_disease(self, image_path: str, top_k: int = 3) -> Dict:
        img = Image.open(image_path).convert("RGB")
        img_tensor = self.transform(img).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            logits = self.cnn_model(img_tensor)
            probabilities = F.softmax(logits, dim=1)[0]
        
        top_probs, top_indices = torch.topk(
            probabilities, k=min(top_k, len(self.class_names))
        )
        
        top_predictions = [
            (self.class_names[idx.item()], prob.item()) 
            for idx, prob in zip(top_indices, top_probs)
        ]
        
        return {
            'top_prediction': top_predictions[0][0],
            'confidence': top_predictions[0][1],
            'top_k_predictions': top_predictions,
            'is_confident': top_predictions[0][1] >= self.confidence_threshold
        }
    

    def diagnose_and_advise(self, image_path: str, user_question: Optional[str] = None, include_alternatives: bool = True) -> Dict:
        
        prediction = self.predict_disease(image_path)
        primary_disease = prediction['top_prediction']
        
        if user_question:
            query = f"{user_question} (Detected disease: {primary_disease})"
        else:
            query = f"What are the symptoms, causes, and complete treatment plan for {primary_disease}?"
        
        advice = self.rag.query(query)
        
        result = {
            'detection': prediction,
            'primary_advice': advice,
            'alternative_advice': []
        }
        
        if include_alternatives and not prediction['is_confident']:
            for disease, conf in prediction['top_k_predictions'][1:]:
                alt_advice = self.rag.query(f"Brief treatment for {disease}")
                result['alternative_advice'].append({
                    'disease': disease,
                    'confidence': conf,
                    'advice': alt_advice
                })
        
        return result
    

    def print_diagnosis_report(self, result: Dict):
        detection = result['detection']
        advice = result['primary_advice']
        
        print("\n🦠 DETECTED DISEASE:", detection['top_prediction'])
        print("Confidence:", f"{detection['confidence']:.2%}")
        
        if not detection['is_confident']:
            print("\n⚠ LOW CONFIDENCE WARNING")
            for disease, conf in detection['top_k_predictions'][1:3]:
                print(f"Alternative: {disease} ({conf:.2%})")
        
        print("\n💊 TREATMENT:")
        print(advice['answer'])


In [9]:
system = IntegratedPlantDiseaseSystem(
    cnn_model_path="checkpoints/efficientnet_b0_best.pth",
    rag_system=rag,
    confidence_threshold=0.75
)

result = system.diagnose_and_advise(
    image_path=r"K:\cnndisease\00b1f292-23dd-44d4-aad3-c1ffb6a6ad5a___RS_LB 4479.JPG",
    user_question="How can I treat this organically?",
    include_alternatives=True
)

system.print_diagnosis_report(result)


Loaded pretrained weights for efficientnet-b0
✅ Integrated system initialized
CNN Classes: 15
Confidence Threshold: 0.75
Device: cuda

🦠 DETECTED DISEASE: Potato___Late_blight
Confidence: 91.20%

💊 TREATMENT:
Based on the provided knowledge base, the disease you are dealing with is Potato Late Blight.

Here's the information you requested:

**Potato Late Blight:**

- **Symptoms:** Dark water-soaked lesions on leaves, white mold under leaves, tuber rot
- **Cause:** Phytophthora infestans
- **Organic Treatment:** Copper oxychloride, neem oil, baking soda spray
- **Chemical Treatment:** Metalaxyl, Mancozeb, Chlorothalonil
- **Prevention:** Crop rotation, remove infected plants, resistant varieties


In [10]:
import os
print(os.listdir(r"K:\plant villagee\PlantVillage"))


['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']


In [11]:
# ============================
# ✅ FINAL CNN EVALUATION + READINESS CHECK (USING FULL DATASET)
# ============================

from sklearn.metrics import classification_report
import numpy as np
import json
import torch
from efficientnet_pytorch import EfficientNet
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# -------------------------------------------------
# ✅ STEP 0: SET YOUR REAL DATASET PATH ✅ (CONFIRMED)
# -------------------------------------------------

TEST_DIR = r"K:\plant villagee\PlantVillage"   # ✅ YOUR CONFIRMED REAL PATH


# -------------------------------------------------
# ✅ STEP 1: LOAD CLASS NAMES + MODEL
# -------------------------------------------------

checkpoint = torch.load("checkpoints/efficientnet_b0_best.pth", map_location="cuda")

class_names = checkpoint["classes"]
num_total_classes = len(class_names)

print("✅ Total CNN Classes:", num_total_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EfficientNet.from_pretrained(
    "efficientnet-b0",
    num_classes=num_total_classes
)

model.load_state_dict(checkpoint["model"])
model.to(device)
model.eval()

print("✅ CNN model rebuilt and loaded")


# -------------------------------------------------
# ✅ STEP 2: BUILD DATA LOADER FROM YOUR DATASET
# -------------------------------------------------

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

full_dataset = datasets.ImageFolder(TEST_DIR, transform=transform)
test_loader = DataLoader(full_dataset, batch_size=32, shuffle=False)

print("✅ Dataset Loaded:", TEST_DIR)
print("✅ Total Images Found:", len(full_dataset))


# -------------------------------------------------
# ✅ STEP 3: GENERATE y_true & y_pred
# -------------------------------------------------

y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print("✅ Unique Labels Found:", len(np.unique(y_true)))


# -------------------------------------------------
# ✅ STEP 4: GENERATE CLASSIFICATION REPORT
# -------------------------------------------------

report = classification_report(
    y_true,
    y_pred,
    labels=np.unique(y_true),
    target_names=[class_names[i] for i in np.unique(y_true)],
    output_dict=True,
    zero_division=0
)


# -------------------------------------------------
# ✅ STEP 5: CNN READINESS FUNCTION
# -------------------------------------------------

def check_model_readiness(classification_report_dict):
    
    overall_accuracy = classification_report_dict.get("accuracy", 0)
    macro_f1 = classification_report_dict.get("macro avg", {}).get("f1-score", 0)
    
    poor_classes = []
    for class_name, metrics in classification_report_dict.items():
        if isinstance(metrics, dict) and "f1-score" in metrics:
            if metrics["f1-score"] < 0.75:
                poor_classes.append((class_name, metrics["f1-score"]))
    
    if overall_accuracy >= 0.95 and macro_f1 >= 0.93:
        status = "✅ EXCELLENT"
    elif overall_accuracy >= 0.90:
        status = "✓ GOOD"
    elif overall_accuracy >= 0.85:
        status = "⚠ ACCEPTABLE"
    else:
        status = "❌ NOT READY"
    
    return {
        "status": status,
        "overall_accuracy": overall_accuracy,
        "macro_f1": macro_f1,
        "poor_performing_classes": poor_classes
    }


# -------------------------------------------------
# ✅ STEP 6: RUN READINESS CHECK
# -------------------------------------------------

readiness = check_model_readiness(report)

print("\n==============================")
print("✅ CNN DEPLOYMENT READINESS")
print("==============================")

print("MODEL STATUS:", readiness["status"])
print("Overall Accuracy:", round(readiness["overall_accuracy"] * 100, 2), "%")
print("Macro F1 Score:", round(readiness["macro_f1"], 4))

if readiness["poor_performing_classes"]:
    print("\n⚠ WEAK CLASSES (Need Improvement):")
    for cls, f1 in readiness["poor_performing_classes"]:
        print(f"  {cls} → F1 = {f1:.3f}")
else:
    print("\n✅ ALL CLASSES ARE ABOVE MINIMUM F1 THRESHOLD")


# -------------------------------------------------
# ✅ STEP 7: SAVE REPORT
# -------------------------------------------------

with open("cnn_readiness_report.json", "w") as f:
    json.dump(readiness, f, indent=4)

print("\n✅ Readiness report saved as: cnn_readiness_report.json")


✅ Total CNN Classes: 15
Loaded pretrained weights for efficientnet-b0
✅ CNN model rebuilt and loaded
✅ Dataset Loaded: K:\plant villagee\PlantVillage
✅ Total Images Found: 20638


KeyboardInterrupt: 